In [1]:
import os
import time
from IPython.display import clear_output
from collections import defaultdict, Counter
import math
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import numpy as np
import copy
import random
import pickle
import warnings
warnings.filterwarnings("ignore")

In [2]:
# must have n lines
# read in contents of single file as dictionary
# load all files as idct of dicts
# calculate tf-idf for each word
# get top n 

#sparse matrix representation

In [3]:
stop_words = set(stopwords.words('english'))

## Read in files to dictionary

In [4]:
src_dir = '/Users/choldawa/Documents/Projects/RedditCorpus/word_count/RC_2012-12'

tick = time.time()

d = {}
problem_subs = []
cnt = 0
for file in os.listdir(src_dir):
    path = src_dir+"/"+file
    with open(path) as f:
        filename = os.path.basename(file).split("_")[2].split(".")[0]
        clear_output(wait=True)
        print(f"working on: {filename}")
        tock = time.time()
        print(f"Time elapsed: {tock - tick} seconds")
        sub_d = {}
        num_lines = sum(1 for line in open(path)) #must have at least 100 unique words
        if num_lines > 100:
            for line in f:
                try:
                    (val, key) = line.strip().split() #invert the order 
                    if not any(map(str.isdigit, key)) and int(val)>1: # each word must occur more than once
                         if key not in stop_words: #check if stopword   
                            sub_d[key] = int(val)
                except:
                    problem_subs.append(filename)
            d[filename] = sub_d
end = time.time()
print(f"Total Time elapsed: {end - tick} seconds")
print(f"We have {len(d)} Subreddits")
print(f"Found problems with {len(problem_subs)} subreddits") 

working on: CrowdfundedBoardgames
Time elapsed: 96.46595406532288 seconds
Total Time elapsed: 96.46697521209717 seconds
We have 11427 Subreddits
Found problems with 122 subreddits


In [5]:
print("Here are some example first and last items:")
print("Common words:")
print(list(d['politics'].items())[:5])
print("Uncommon words:")
print(list(d['politics'].items())[-20:])

Here are some example first and last items:
Common words:
[('people', 116175), ('would', 84858), ('dont', 77698), ('like', 70930), ('one', 56556)]
Uncommon words:
[('abccbsnbc', 2), ('abbreviations', 2), ('abbrasive', 2), ('abbottabad', 2), ('abbotsford', 2), ('abbos', 2), ('abbiex', 2), ('abbasled', 2), ('abating', 2), ('abated', 2), ('aardvarks', 2), ('aah', 2), ('aaguns', 2), ('aaah', 2), ('aaaah', 2), ('aaaaannndd', 2), ('aaaaaaaaaand', 2), ('aaaaaaaaaaand', 2), ('aaaaaaaaaaaand', 2), ('á', 2)]


In [6]:
N = len(d)

df = defaultdict(int)
for s in d:
    for w in d[s]: #count of subs where w appears
        df[w] += 1

In [7]:
df['crypto']

56

## SKLEARN tf-idf approach

In [8]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
dv = DictVectorizer()
#D = [{'foo': 1, 'bar': 2}, {'foo': 3, 'baz': 1}] #EXAMPLE FORMAT
X = dv.fit_transform(list(d.values()))
tv = TfidfTransformer()
tfidf = tv.fit_transform(X)
print(tfidf.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
tfidf.toarray().shape

(11427, 652887)

In [10]:
k = 5
# Define the type of our output array elements: (int, float)
dt = np.dtype([('index', np.int32, 1), ('value', np.float64, 1)])

# Take the indices of the largest k elements from each row
top_k_inds = np.argsort(tfidf.toarray())[:, -1:-k - 1:-1]

# Take the values at those indices
top_k = np.take_along_axis(tfidf.toarray(), top_k_inds, axis=-1)

# Stack the two together along a third axis (to get index-value pairs)
top_k_pairs = np.stack((top_k_inds, top_k), axis=2)

# Convert the type (otherwise we have the indices as floats)
# top_k_pairs = top_k_pairs.astype(dt)

In [11]:
top_k_pairs[1]

array([[3.98118000e+05, 3.93636720e-01],
       [6.47480000e+04, 1.93462826e-01],
       [1.26528000e+05, 1.83498888e-01],
       [4.18186000e+05, 1.57209444e-01],
       [3.89239000e+05, 1.43874628e-01]])

In [12]:
for i in range(len(top_k_pairs[22])):
    print(dv.get_feature_names()[int(top_k_pairs[22][i][0])])

teamfight
void
earlygame
ult
sven


In [26]:
list(d.keys()).index('wallstreetbets')

6172

In [50]:
subreddit = 'wallstreetbets' #wallstreetbets #crypto #nba #Conservative
index = list(d.keys()).index(subreddit)
print(f'Subreddit: r/{subreddit}')
print()
print('top content by count:')
print(list(d[list(d.keys())[index]])[:10])
print()
print(f'top tfidf for r/{subreddit}:')
for i in range(len(top_k_pairs[index])):
    print(dv.get_feature_names()[int(top_k_pairs[index][i][0])])
#dv.get_feature_names()[int(top_k_pairs[index][0][0])]

Subreddit: r/wallstreetbets

top content by count:
['think', 'im', 'get', 'approval', 'heb', 'time', 'short', 'adcom', 'good', 'dont']

top tfidf for r/wallstreetbets:
adcom
heb
approval
pdufa
ampligen


In [29]:
df['the']/len(d)

0.0

In [ ]:
#list(d.keys()) # rowNames
#list(d.values()) #dictionary input to tfidf
# dv.get_feature_names() #colNames
# list(set(key for dic in D for key in dic.keys())) #colNames

#get topN
#feature_array = np.array(tfidf.get_feature_names())
# tfidf_sorting = np.argsort(response.toarray()).flatten()[::-1]
# n = 3
# top_n = feature_array[tfidf_sorting][:n]


## TF-IDF Calculations (our way)

In [ ]:
sub_dict = copy.deepcopy(d) #make a deepcopy (need to copy nested obj)

In [ ]:
N = len(sub_dict)

df = defaultdict(int)
for s in sub_dict:
    for w in sub_dict[s]: #count of subs where w appears
        df[w] += 1
print(f" We have {N} sub_reddits and {df['game']} contain 'game'")

In [ ]:
for s in sub_dict:
    for key, value in sub_dict[s].items():
        tf = (value/
                  ((sum(sub_dict[s].values())-value)+1))
        idf = (np.log(N / df[key]))
        tfidf = tf*idf
        sub_dict[s][key] = tfidf
    

In [ ]:
for s in sub_dict:
    print(s, 
          list(dict(sorted(sub_dict[s].items(), 
                           key=lambda item: item[1], reverse = True)
                    [:3]).items()))

In [ ]:

with open('12_2012_tfidf.pickle', 'wb') as handle:
    pickle.dump(sub_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('12_2012_tfidf.pickle', 'rb') as handle:
    sub_dict = pickle.load(handle)

### Plot histogram of word counts

In [ ]:
print(f"We have {len(df)} unique words and {len(d)} unique subreddits")
logD = {k:np.log10(v) for k, v in df.items()}
plt.hist(logD.values(), bins = 25)
plt.xlabel("Log10 num of subreddits")
plt.ylabel("Count of words")
plt.plot()

### Plotting with Counter (not sure if it works with non-integers...)

In [ ]:
# sub_reddit = 'america' #america #politics # theoffice
# topN = 10
# c = Counter(sub_dict[sub_reddit]) #create counter

# most_common = c.most_common(topN) # get topN tf-idf
# most_common

# plt.bar(range(len(most_common)), [val[1] for val in most_common], align='center')
# plt.xticks(range(len(most_common)), [val[0] for val in most_common])
# plt.xticks(rotation=70)
# plt.title(sub_reddit)
# plt.show()

### Plotting sorted dict

In [ ]:
sub_reddit = 'theoffice' # theoffice
topN = 6
x = list(dict(sorted(sub_dict[sub_reddit].items(), 
                           key=lambda item: item[1], reverse = True)
                    [:topN]).keys())
y = list(dict(sorted(sub_dict[sub_reddit].items(), 
                           key=lambda item: item[1], reverse = True)
                    [:topN]).values())


plt.bar(x,y, align='center')
plt.xticks(rotation=70)
plt.title(sub_reddit)
plt.show()

In [ ]:
d = [{"sub_1":{'foo': 1, "bar":2}}, 
     {"sub_2": {'foo':3, "baz": 10, 'bar':2}}]

#take a list of dictionaries with dictionaries
#run sklearn tf-idf to get matrix
# access top N tf-df for any row 
# get the names of those words (with highest tf-idf)
# tie that row back to the name of the subreddit

d[0]